# Going From Staregies A&B To A Portfolio

So far, I have implemented two startegies using momentum and news analytics on Quantopian along with their backtests. The universe of assets is considered to be Q1500US - the 1500 most liquid equities in the US stock exchange. In this notebook I will attempt to take the best performing equities from those startegies for my portfolio and size them appropriately

## Background Work

In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
from IPython.display import display

pd.options.display.max_rows = 5

### Defining All Equities

In [5]:
equities_strategyA = [
    'ARNC',
    'ABT',
    'ADSK',
    'ADBE',
    'ADI',
    'AEP',
    'AES',
    'AFL',
    'AMD',
    'ADP'
]

equities_strategyB = [
    'ENDP',
    'MDCO',
    'WW',
    'STMP',
    'INFN',
    'SEMG',
    'MIK',
    'RETA',
    'CLDR',
    'HOME',
]

### Querying Data From Yahoo Finance

In [6]:
end = datetime.now().strftime("%Y-%m-%d")
start = datetime.now() - relativedelta(years=1)


data_strategyA = yf.download(equities_strategyA,
                             start,
                             end)['Close']
data_strategyB = yf.download(equities_strategyB,
                             start,
                             end)['Close']

print("\nFields:", data_strategyA.columns.get_level_values(0).unique())

display(data_strategyA)
display(data_strategyB)

data_strategyA.plot(figsize=(15,5)); plt.grid(); plt.title('StrategyA')
data_strategyB.plot(figsize=(15,5)); plt.grid(); plt.title('StrategyB')

[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed

Fields: Index(['ABT', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AEP', 'AES', 'AFL', 'AMD',
       'ARNC'],
      dtype='object')


,ABT,ADBE,ADI,ADP,ADSK,AEP,AES,AFL,AMD,ARNC
Date,,,,,,,,,,
2018-11-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-29,73.260002,249.089996,89.519997,145.850006,145.550003,76.349998,15.440000,45.400002,21.430000,21.340000
...,...,...,...,...,...,...,...,...,...,...
2019-11-26,85.419998,307.899994,114.889999,171.619995,170.800003,91.500000,18.750000,54.730000,38.990002,31.290001
2019-11-27,85.419998,309.059998,113.699997,171.479996,180.179993,91.589996,18.889999,54.820000,39.410000,31.260000


,CLDR,ENDP,HOME,INFN,MDCO,MIK,RETA,SEMG,STMP,WW
Date,,,,,,,,,,
2018-11-29,12.35,12.15,27.709999,4.37,21.450001,17.090000,63.400002,16.280001,170.259995,51.250000
2018-11-30,12.34,12.03,28.500000,4.31,22.129999,16.969999,63.150002,16.230000,171.460007,50.020000
...,...,...,...,...,...,...,...,...,...,...
2019-11-26,9.76,4.62,8.570000,6.42,83.879997,8.180000,199.050003,15.450000,89.150002,43.389999
2019-11-27,9.95,4.78,8.570000,6.47,84.050003,8.200000,195.710007,15.400000,88.900002,44.290001


### Volatility Calculation

In [7]:
volatilityA = data_strategyA.rolling(22).std().dropna(how='all')
volatilityB = data_strategyB.rolling(22).std().dropna(how='all')

display(volatilityA)
display(volatilityB)

volatilityA.plot(figsize=(15,5)); plt.grid(); plt.title('StrategyA')
volatilityB.plot(figsize=(15,5)); plt.grid(); plt.title('StrategyB')

,ABT,ADBE,ADI,ADP,ADSK,AEP,AES,AFL,AMD,ARNC
Date,,,,,,,,,,
2019-01-02,2.098381,14.537389,3.185301,7.410283,7.854401,2.208197,0.664255,1.124584,1.724603,1.751920
2019-01-03,2.226695,14.538458,3.402288,7.265215,7.580474,2.398800,0.683810,1.096895,1.733212,1.676804
...,...,...,...,...,...,...,...,...,...,...
2019-11-26,0.868823,10.602460,2.319974,4.468545,7.773600,1.772200,0.629351,0.439323,2.478344,1.256755
2019-11-27,0.898435,10.404885,2.301155,4.542952,8.893375,1.708667,0.621717,0.471482,2.400075,1.231198


,CLDR,ENDP,HOME,INFN,MDCO,MIK,RETA,SEMG,STMP,WW
Date,,,,,,,,,,
2019-01-02,0.789772,1.920738,3.966291,0.266215,1.751899,1.472586,4.818303,1.076178,9.976288,4.350722
2019-01-03,0.824446,1.891282,3.656077,0.259816,1.756616,1.377411,4.768085,1.059908,9.981389,4.551310
...,...,...,...,...,...,...,...,...,...,...
2019-11-26,0.356726,0.238079,0.383889,0.555649,10.096145,0.662534,7.814457,0.503745,2.665119,3.297759
2019-11-27,0.420842,0.245794,0.317564,0.554211,11.329288,0.587983,8.038263,0.503917,2.348886,3.638282


### Getting Asset Correlation (from Close Price)

In [8]:
corrA = data_strategyA.corr()
corrB = data_strategyB.corr()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(corrA)
    display(corrB)

,ABT,ADBE,ADI,ADP,ADSK,AEP,AES,AFL,AMD,ARNC
ABT,1.000000,0.902990,0.842600,0.889105,0.445956,0.875474,0.321091,0.847517,0.866608,0.784204
ADBE,0.902990,1.000000,0.851734,0.912957,0.692775,0.791036,0.415528,0.876598,0.880356,0.744233
ADI,0.842600,0.851734,1.000000,0.851116,0.684805,0.734566,0.540787,0.751313,0.760176,0.583635
ADP,0.889105,0.912957,0.851116,1.000000,0.658273,0.854997,0.432700,0.827290,0.877254,0.712378
ADSK,0.445956,0.692775,0.684805,0.658273,1.000000,0.336907,0.664484,0.603958,0.509546,0.210205
AEP,0.875474,0.791036,0.734566,0.854997,0.336907,1.000000,0.207273,0.824863,0.850536,0.853352
AES,0.321091,0.415528,0.540787,0.432700,0.664484,0.207273,1.000000,0.413802,0.421395,0.133855
AFL,0.847517,0.876598,0.751313,0.827290,0.603958,0.824863,0.413802,1.000000,0.859601,0.747368
AMD,0.866608,0.880356,0.760176,0.877254,0.509546,0.850536,0.421395,0.859601,1.000000,0.892595
ARNC,0.784204,0.744233,0.583635,0.712378,0.210205,0.853352,0.133855,0.747368,0.892595,1.000000


,CLDR,ENDP,HOME,INFN,MDCO,MIK,RETA,SEMG,STMP,WW
CLDR,1.000000,0.875685,0.885125,0.298699,-0.475742,0.835306,-0.141406,0.629982,0.718996,0.181552
ENDP,0.875685,1.000000,0.915079,-0.035221,-0.682256,0.926747,-0.294810,0.581720,0.754803,0.125949
HOME,0.885125,0.915079,1.000000,-0.063444,-0.652685,0.858101,-0.331178,0.502419,0.570451,-0.067202
INFN,0.298699,-0.035221,-0.063444,1.000000,0.512445,-0.006139,0.539788,0.355027,0.238829,0.494697
MDCO,-0.475742,-0.682256,-0.652685,0.512445,1.000000,-0.627607,0.762978,-0.058288,-0.468416,0.198548
MIK,0.835306,0.926747,0.858101,-0.006139,-0.627607,1.000000,-0.282937,0.687788,0.773882,0.248680
RETA,-0.141406,-0.294810,-0.331178,0.539788,0.762978,-0.282937,1.000000,0.242436,-0.178185,0.180671
SEMG,0.629982,0.581720,0.502419,0.355027,-0.058288,0.687788,0.242436,1.000000,0.589156,0.390322
STMP,0.718996,0.754803,0.570451,0.238829,-0.468416,0.773882,-0.178185,0.589156,1.000000,0.558344
WW,0.181552,0.125949,-0.067202,0.494697,0.198548,0.248680,0.180671,0.390322,0.558344,1.000000


## Using Calculations Above For Position Sizing

### Startegy A

In [9]:
volatilityMeanAdjusted = volatilityA.mean()/data_strategyA.mean()
volatilityMeanAdjusted = (volatilityMeanAdjusted)/volatilityMeanAdjusted.sum()


corrAdjusted = corrA.sum(axis=1) - 1
corrAdjusted = (corrAdjusted-corrAdjusted.min())/corrAdjusted.sum()

position_weightsA = 1/(volatilityMeanAdjusted+corrAdjusted)
position_weightsA = position_weightsA/position_weightsA.sum()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(position_weightsA*100)

ABT     10.700460
ADBE     9.029134
ADI      7.913862
ADP     10.520589
ADSK     9.193768
AEP     13.392930
AES     13.637357
AFL     11.548869
AMD      5.630541
ARNC     8.432490
dtype: float64


### Streategy B

In [10]:
volatilityMeanAdjusted = volatilityB.mean()/data_strategyB.mean()
volatilityMeanAdjusted = (volatilityMeanAdjusted)/volatilityMeanAdjusted.sum()


corrAdjusted = corrB.sum(axis=1) - 1
corrAdjusted = (corrAdjusted-corrAdjusted.min())/corrAdjusted.sum()

position_weightsB = 1/(volatilityMeanAdjusted+corrAdjusted)
position_weightsB = position_weightsB/position_weightsB.sum()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(position_weightsB*100)

CLDR     6.949129
ENDP     6.760549
HOME     7.337295
INFN     8.938009
MDCO    30.477180
MIK      7.151606
RETA    10.846036
SEMG     6.992850
STMP     6.137632
WW       8.409715
dtype: float64

## Allocating Risk Based On Weights Calulated

So far we have weights (that sum up to 1) for each strategy. How do we decide how much capital to put into each strategy? Here I will allocate 1/2 of the risk into each strategy and size the entire portfolio accordingly. I will be working with the assumption that the max drawdown for both the strategies combined is 10M. Based on a 50% risk allocation, this will mean I need to size each strategy's positions such the drawdown for the strategy does not exceed 5M.

### Strategy A

In [11]:
df_A = pd.concat([data_strategyA.iloc[-1, :], position_weightsA, volatilityA.mean()],
          axis=1)
df_A.columns = ['Price Today', 'Weight', 'Avg Volatility']
df_A['-2std'] = -2 * df_A['Avg Volatility']
df_A['Risk'] = 5e5 * df_A['Weight']
df_A['Shares'] = -(df_A['Risk']/df_A['-2std']).round()
df_A['Value Today'] = df_A['Shares'] * df_A['Price Today']

print('Total Dollars Invested Today: ', df_A['Value Today'].sum())


with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_A)

Total Dollars Invested Today:  12216569.72816658


,Price Today,Weight,Avg Volatility,-2std,Risk,Shares,Value Today
ABT,85.419998,0.107005,1.578029,-3.156059,53502.301350,16952.0,1.448040e+06
ADBE,309.059998,0.090291,6.794914,-13.589827,45145.669321,3322.0,1.026697e+06
ADI,113.699997,0.079139,3.483700,-6.967400,39569.311382,5679.0,6.457023e+05
ADP,171.479996,0.105206,3.048585,-6.097169,52602.943002,8627.0,1.479358e+06
ADSK,180.179993,0.091938,5.317463,-10.634926,45968.840758,4322.0,7.787379e+05
AEP,91.589996,0.133929,1.290904,-2.581808,66964.648430,25937.0,2.375570e+06
AES,18.889999,0.136374,0.446913,-0.893825,68186.784903,76286.0,1.441042e+06
AFL,54.820000,0.115489,0.887099,-1.774197,57744.344404,32547.0,1.784227e+06
AMD,39.410000,0.056305,1.405314,-2.810629,28152.704794,10017.0,3.947700e+05
ARNC,31.260000,0.084325,0.782254,-1.564509,42162.451656,26949.0,8.424257e+05


### Strategy B

In [12]:
df_B = pd.concat([data_strategyB.iloc[-1, :], position_weightsB, volatilityB.mean()],
          axis=1)
df_B.columns = ['Price Today', 'Weight', 'Avg Volatility']
df_B['-2std'] = -2 * df_B['Avg Volatility']
df_B['Risk'] = 5e5 * df_B['Weight']
df_B['Shares'] = -(df_B['Risk']/df_B['-2std']).round()
df_B['Value Today'] = df_B['Shares'] * df_B['Price Today']

print('Total Dollars Invested Today: ', df_B['Value Today'].sum())


with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_B)

Total Dollars Invested Today:  6063904.31189394


,Price Today,Weight,Avg Volatility,-2std,Risk,Shares,Value Today
CLDR,9.950000,0.069491,0.651831,-1.303663,34745.646998,26652.0,2.651874e+05
ENDP,4.780000,0.067605,0.590377,-1.180754,33802.742779,28628.0,1.368418e+05
HOME,8.570000,0.073373,1.383986,-2.767972,36686.475109,13254.0,1.135868e+05
INFN,6.470000,0.089380,0.277601,-0.555202,44690.044598,80493.0,5.207897e+05
MDCO,84.050003,0.304772,1.918509,-3.837018,152385.900117,39715.0,3.338046e+06
MIK,8.200000,0.071516,0.782105,-1.564209,35758.029800,22860.0,1.874520e+05
RETA,195.710007,0.108460,8.556646,-17.113293,54230.179159,3169.0,6.202050e+05
SEMG,15.400000,0.069928,0.877142,-1.754284,34964.247510,19931.0,3.069374e+05
STMP,88.900002,0.061376,9.713965,-19.427930,30688.159495,1580.0,1.404620e+05
WW,44.290001,0.084097,2.143667,-4.287334,42048.574434,9808.0,4.343963e+05


## Portfolio (Optimized) 

In [13]:
portfolio = pd.concat([df_A[['Price Today', 'Shares', 'Value Today']], 
         df_B[['Price Today', 'Shares', 'Value Today']]],
         axis=0)
portfolio.columns = ['Price Today', 'Optimal Shares', 'Value Today']

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(portfolio)

,Price Today,Optimal Shares,Value Today
ABT,85.419998,16952.0,1.448040e+06
ADBE,309.059998,3322.0,1.026697e+06
ADI,113.699997,5679.0,6.457023e+05
ADP,171.479996,8627.0,1.479358e+06
ADSK,180.179993,4322.0,7.787379e+05
AEP,91.589996,25937.0,2.375570e+06
AES,18.889999,76286.0,1.441042e+06
AFL,54.820000,32547.0,1.784227e+06
AMD,39.410000,10017.0,3.947700e+05
ARNC,31.260000,26949.0,8.424257e+05


In [16]:
portfolio['Value Today'].sum()/1e6

18.280474040060522

## What About the Current Holdings?

At this point we have determined optimal position sizes based on the risk allocation from weights above. However, at the time, I have already submitted multiple trades. In this section I will investigate wheather the worst case, for the current risk allocation, takes me less than 1M.

In [18]:
current_B = pd.DataFrame([95480,
8700,
12880,
5400,
79540,
27880,
52900,
2180,
52840,
51780],
                        index = equities_strategyB,
                        columns = ['Shares Holding'])

current_A = pd.DataFrame([15860,
6160,
2520,
2720,
3620,
3620,
21800,
6760,
14840,
2600], 
                        index = equities_strategyA,
                        columns = ['Shares Holding'])

current_A['Risk'] = df_A['-2std']*current_A['Shares Holding']
current_B['Risk'] = df_B['-2std']*current_B['Shares Holding']

In [19]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(current_A)
    display(current_B)
    
print('Worst Case Drawdown:', current_A['Risk'].sum() + current_B['Risk'].sum())

,Shares Holding,Risk
ARNC,15860,-24813.111700
ABT,6160,-19441.322867
ADSK,2520,-26800.013499
ADBE,2720,-36964.329654
ADI,3620,-25221.989594
AEP,3620,-9346.143295
AES,21800,-19485.389721
AFL,6760,-11993.571920
AMD,14840,-41709.730617
ADP,2600,-15852.640403


,Shares Holding,Risk
ENDP,95480,-112738.388906
MDCO,8700,-33382.060454
WW,12880,-55220.857262
STMP,5400,-104910.822245
INFN,79540,-44160.751909
SEMG,27880,-48909.444382
MIK,52900,-82746.662209
RETA,2180,-37306.977959
CLDR,52840,-68885.531322
HOME,51780,-143325.584418


Worst Case Drawdown: -963215.3243367152


I'm barely meeting the criteria - I have my current (suboptimally weighted) portfolio structure such that the worst case drawdown is less than -1M. So for now I will keep my positions.